## Limpieza de la data

#### Primero vamos a elmininar la copia.db

In [2]:
import os

# Nombre del archivo de la base de datos
db_file = "../data/processed/copia.db"

# Verificar si existe y eliminarlo
if os.path.exists(db_file):
    os.remove(db_file)
    print("Base de datos eliminada exitosamente.")
else:
    print("El archivo de la base de datos no existe.")

Base de datos eliminada exitosamente.


In [3]:
import pandas as pd
import sqlite3

In [4]:
# Cargar datos
df = pd.read_csv("../data/raw/ufo_data.csv")
print("✅ Datos cargados")

✅ Datos cargados


C:\Users\harri\AppData\Local\Temp\ipykernel_14340\2705268744.py:2: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/ufo_data.csv")


In [5]:
# Crearemos una conexión sqlite3
conexion = sqlite3.connect("../data/raw/ufo_data.db")

# Guardar el dataframe como tabla SQLite
df.to_sql("ufo_table", conexion, if_exists="replace", index=False) 
# El index col se usa para eliminar el Unnamed

print("✅ Base de datos creada correctamente...")

cursor = conexion.cursor()

✅ Base de datos creada correctamente...


In [6]:
# Crear una copia de la db
copia = "../data/processed/copia.db"

consulta = f"""
ATTACH DATABASE '{copia}' AS copia;
SELECT sql FROM sqlite_master WHERE type='table';
"""
cursor.executescript(consulta)

# Copiar las tablas a la base de datos de respaldo
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablas = cursor.fetchall()

for tabla in tablas:
    nombre_tabla = tabla[0]
    consulta_copia = f"CREATE TABLE copia.{nombre_tabla} AS SELECT * FROM {nombre_tabla};"
    cursor.execute(consulta_copia)

print("✅ Copia de la BD creada correctamente...")
# Ya no volver a ejecutar esto
# Si lo hacemos primero hay que cerrar la conexión
# Y luego hay que eliminar el archivo copia.db

✅ Copia de la BD creada correctamente...


Lo que haremos en esta fase:

2. Ahora si procedemos hacer la limpieza:

    ✅ Datos faltantes (Nulos).
    
    ✅ Registros duplicados.
    
    ✅ Formatos inconsistentes (fechas, nombres, números).
    
    ✅ Valores atípicos (outliers).

### ✅ Manejar Datos Faltantes (Nulos)

In [7]:
# Reemplazar Nulos
consulta = """
           UPDATE copia.ufo_table
           SET
               state = COALESCE(state, "Desconocido"),
               country = COALESCE(country, "Desconocido"),
               shape = COALESCE(shape, "No especificado"),
               comments = COALESCE(comments, "Sin comentarios")
           """
conexion.execute(consulta)
conexion.commit()
print("Valores reemplazados con éxito")

Valores reemplazados con éxito


In [8]:
nombre_columnas = "PRAGMA table_info(ufo_table);"
nombre_columnas = pd.read_sql_query(nombre_columnas, conexion)
nombre_columnas

,cid,name,type,notnull,dflt_value,pk
0,0,datetime,TEXT,0,None,0
1,1,city,TEXT,0,None,0
2,2,state,TEXT,0,None,0
3,3,country,TEXT,0,None,0
4,4,shape,TEXT,0,None,0
5,5,duration (seconds),TEXT,0,None,0
6,6,duration (hours/min),TEXT,0,None,0
7,7,comments,TEXT,0,None,0
8,8,date posted,TEXT,0,None,0
9,9,latitude,TEXT,0,None,0


### ✅ Manejar registros duplicados

In [9]:
# Eliminar duplicados
eliminar = """
           WITH cte AS (
                SELECT ROWID, ROW_NUMBER() OVER(PARTITION BY datetime, city, state, country, shape) AS num_fila
                FROM copia.ufo_table
           )
           DELETE FROM copia.ufo_table
           WHERE ROWID IN (
               SELECT ROWID FROM cte WHERE num_fila > 1
           );
           """
conexion.execute(eliminar)
conexion.commit()
print("Duplicados eliminados con éxito")

Duplicados eliminados con éxito


### ✅ Normalizar los tipos de datos (Fechas, numeros)

In [10]:
# Convertir datetime de TEXT a DATETIME(ya que es fecha y hora, si fuera solo fecha sería DATE y si solo fuera hora sería TIME)
# Convertir duration seconds de TEXT a REAL
formatear = """
            UPDATE copia.ufo_table
            SET `duration (seconds)` = CAST(`duration (seconds)` AS REAL)
            ;
            """
conexion.execute(formatear)
conexion.commit()
print("Datos normalizados con éxito")

Datos normalizados con éxito


In [11]:
# Verificando si se hicieron los cambios correctamente
consulta_db = "SELECT typeof(`duration (seconds)`) FROM copia.ufo_table LIMIT 10;"
consulta_db = pd.read_sql_query(consulta_db, conexion)
consulta_db

,typeof(`duration (seconds)`)
0,text
1,text
2,text
3,text
4,text
5,text
6,text
7,text
8,text
9,text


#### Como no afectan los cambios a la tabla, vamos a crear una nueva variable o columna duration_seconds y copiar los datos

In [12]:
# Creamos una nueva columna
nueva_columna = "ALTER TABLE copia.ufo_table ADD COLUMN duration_seconds REAL;"

conexion.execute(nueva_columna)
conexion.commit()

In [13]:
# Copiamos los valores de 'duration (seconds)' a duration_seconds
copiar_datos = """
            UPDATE copia.ufo_table
            SET duration_seconds = CAST(`duration (seconds)` AS REAL);
            """
conexion.execute(copiar_datos)
conexion.commit()

### Identificando OUTLIERS para durations_seconds

In [14]:
df = pd.read_sql_query("SELECT duration_seconds FROM copia.ufo_table;", conexion)

q1 = df["duration_seconds"].quantile(0.25)
q3 = df["duration_seconds"].quantile(0.75)

iqr = q3 - q1

limite_inferior = q1 - 1.5 * iqr
limite_superior = q3 + 1.5 * iqr

print(f"Q1: {q1}, Q3: {q3}, IQR: {iqr}")
print(f"Límite Inferior: {limite_inferior}, Límite Superior: {limite_superior}")

query = f"""
    SELECT duration_seconds
    FROM copia.ufo_table
    WHERE duration_seconds < {limite_inferior} OR duration_seconds > {limite_superior}
"""
outliers = pd.read_sql_query(query, conexion)
# Mostrar los valores atípicos
print("Total Outliers: ", outliers)

Q1: 30.0, Q3: 600.0, IQR: 570.0
Límite Inferior: -825.0, Límite Superior: 1455.0
Total Outliers:         duration_seconds
0                2700.0
1                7200.0
2                1800.0
3                1800.0
4                2700.0
...                 ...
10400            2100.0
10401            5400.0
10402            7200.0
10403           10800.0
10404            3600.0

[10405 rows x 1 columns]


#### Valores atípicos con la diesviación estandar

In [15]:
# Calcular media y desviación estándar
media = df["duration_seconds"].mean()
std_dev = df["duration_seconds"].std()

# Definir los límites de outliers con 2 desviaciones estándar
limite_inferior = media - 2 * std_dev
limite_superior = media + 2 * std_dev

print(f"Media: {media}, Desviación estándar: {std_dev}")
print(f"Límite Inferior: {limite_inferior}, Límite Superior: {limite_superior}")

# Filtrar los outliers usando estos nuevos límites
query = f"""
    SELECT duration_seconds
    FROM copia.ufo_table
    WHERE duration_seconds < {limite_inferior} OR duration_seconds > {limite_superior}
    ORDER BY duration_seconds ASC;
"""
outliers = pd.read_sql_query(query, conexion)

print(f"Total Outliers con Desviación Estándar: {len(outliers)}")
print(outliers)


Media: 9079.846634475922, Desviación estándar: 622662.6608135117
Límite Inferior: -1236245.4749925474, Límite Superior: 1254405.1682614994
Total Outliers con Desviación Estándar: 43
    duration_seconds
0          1814400.0
1          1814400.0
2          1814400.0
3          1814400.0
4          1814400.0
5          2102400.0
6          2102400.0
7          2419200.0
8          2419200.0
9          2631600.0
10         2631600.0
11         2631600.0
12         2631600.0
13         2631600.0
14         2631600.0
15         2631600.0
16         2631600.0
17         3654000.0
18         4838400.0
19         5263200.0
20         5263200.0
21         6120000.0
22         6312000.0
23         6312000.0
24         6312000.0
25         6312000.0
26         6312000.0
27         6312000.0
28         6312000.0
29         6312000.0
30         7894800.0
31         9468000.0
32         9468000.0
33        10526400.0
34        10526400.0
35        10526400.0
36        25248000.0
37        52623200.0

## ¿Cuál método usar?

✅ IQR (Rango Intercuartílico):

Útil cuando hay datos muy asimétricos o con sesgo.

Puede marcar demasiados outliers si la distribución está sesgada.

✅ Desviación estándar:

Mejor cuando los datos siguen una distribución normal.

Menos sensible a extremos que el IQR.

### ✅ Manejo de Outliers

In [16]:
# Vamos a eliminar los outliers
eliminar_outliers = """
                DELETE FROM copia.ufo_table
                WHERE duration_seconds < (
                    SELECT P25 - 1.5 * IQR FROM (
                        SELECT P25, P75, (P75 - P25) AS IQR FROM (
                            SELECT 
                                (SELECT duration_seconds FROM (
                                    SELECT duration_seconds, ROW_NUMBER() OVER (ORDER BY duration_seconds) AS fila 
                                    FROM copia.ufo_table
                                ) WHERE fila = (SELECT CAST(0.25 * (COUNT(*) - 1) + 1 AS INT) FROM copia.ufo_table)) AS P25,

                                (SELECT duration_seconds FROM (
                                    SELECT duration_seconds, ROW_NUMBER() OVER (ORDER BY duration_seconds) AS fila 
                                    FROM copia.ufo_table
                                ) WHERE fila = (SELECT CAST(0.75 * (COUNT(*) - 1) + 1 AS INT) FROM copia.ufo_table)) AS P75
                        )
                    )
                ) 
                OR 
                duration_seconds > (
                    SELECT P75 + 1.5 * IQR FROM (
                        SELECT P25, P75, (P75 - P25) AS IQR FROM (
                            SELECT 
                                (SELECT duration_seconds FROM (
                                    SELECT duration_seconds, ROW_NUMBER() OVER (ORDER BY duration_seconds) AS fila 
                                    FROM copia.ufo_table
                                ) WHERE fila = (SELECT CAST(0.25 * (COUNT(*) - 1) + 1 AS INT) FROM copia.ufo_table)) AS P25,

                                (SELECT duration_seconds FROM (
                                    SELECT duration_seconds, ROW_NUMBER() OVER (ORDER BY duration_seconds) AS fila 
                                    FROM copia.ufo_table
                                ) WHERE fila = (SELECT CAST(0.75 * (COUNT(*) - 1) + 1 AS INT) FROM copia.ufo_table)) AS P75
                        )
                    )
                );
                """
conexion.execute(eliminar_outliers)
conexion.commit()

In [17]:
total_registros = "SELECT COUNT(*) FROM copia.ufo_table;"
total_registros = pd.read_sql_query(total_registros, conexion)
total_registros

,COUNT(*)
0,69297


In [18]:
# Eliminar outliers para city

eliminar_outliers_city = """
                        DELETE FROM copia.ufo_table
                        WHERE city IN (
                            SELECT city
                            FROM copia.ufo_table
                            GROUP BY city
                            HAVING COUNT(*) < 2
                        );
                         """
conexion.execute(eliminar_outliers_city)
conexion.commit()
print("outliers eliminados correctamente")

outliers eliminados correctamente


In [19]:
total_registros = "SELECT COUNT(*) FROM copia.ufo_table;"
total_registros = pd.read_sql_query(total_registros, conexion)
total_registros

,COUNT(*)
0,57951


# Otros Outliers

- Para country no hay outliers.

- Para shape no hay outliers.

In [ ]:
# Cerrar conexion
#conexion.close()